In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 
import tensorflow as tf
tf.get_logger().setLevel('ERROR')
import pandas as pd
tf.config.set_soft_device_placement(False)
tf.debugging.set_log_device_placement(True)
import numpy as np
from ampligraph.datasets import load_fb15k_237, load_yago3_10
from ampligraph.evaluation.protocol import create_mappings, to_idx

from ampligraph.evaluation.metrics import mrr_score, hits_at_n_score, mr_score
import time
print(tf.__version__)
assert(tf.__version__.startswith('2.3'))

from ampligraph.datasets import load_fb15k_237, load_fb13, load_fb15k, load_wn11, load_wn18, load_wn18rr, load_yago3_10
from ampligraph.latent_features import ScoringBasedEmbeddingModel

2.3.1


In [2]:
#loss function
def nll(scores_pos, scores_neg, eta):
    scores_neg_reshaped = tf.reshape(scores_neg, [eta, tf.shape(scores_pos)[0]])
    neg_exp = tf.exp(scores_neg_reshaped)
    pos_exp = tf.exp(scores_pos)
    softmax_score = pos_exp / (tf.reduce_sum(neg_exp, axis=0) + pos_exp)

    loss = -tf.reduce_sum(tf.math.log(softmax_score))
    return loss

# Train/eval without partition

In [3]:
optim = tf.optimizers.Adagrad(learning_rate=0.1)
# optim = 'adam'

# loss = nll
# loss = 'self_adversarial'
from ampligraph.latent_features.loss_functions import SelfAdversarialLoss
loss = SelfAdversarialLoss({'margin': 0.1, 'alpha': 5})

model = ScoringBasedEmbeddingModel(eta=50, 
                                     k=300,
                                     scoring_type='TransE')



model.compile(optimizer=optim, loss=loss)

from tensorflow.keras.callbacks import ModelCheckpoint
checkpoint = ModelCheckpoint('./chkpt1', monitor='loss', verbose=1, save_best_only=True, mode='min')

start = time.time()
model.fit('/home/spai/code/ampligraph_projects/dataset/fb15k-237/train.txt',
             batch_size=10000,
             epochs=10,
             validation_freq=5,
             validation_batch_size=100,
             validation_data = '/home/spai/code/ampligraph_projects/dataset/fb15k-237/valid.txt',
         callbacks=[checkpoint])

end = time.time()
print('Time taken:', end-start)


Epoch 1/10
     29/Unknown - 1s 51ms/step - loss: 13395.2041
Epoch 00001: loss improved from inf to 13395.20410, saving model to ./chkpt1
29/29 [==============================] - 2s 53ms/step - loss: 13395.2041
Epoch 2/10
29/29 [==============================] - ETA: 0s - loss: 12786.1348
Epoch 00002: loss improved from 13395.20410 to 12786.13477, saving model to ./chkpt1
29/29 [==============================] - 1s 39ms/step - loss: 12786.1348
Epoch 3/10
28/29 [===========================>..] - ETA: 0s - loss: 12628.9609
Epoch 00003: loss improved from 12786.13477 to 12509.62305, saving model to ./chkpt1
29/29 [==============================] - 1s 39ms/step - loss: 12509.6230
Epoch 4/10
28/29 [===========================>..] - ETA: 0s - loss: 12431.5889
Epoch 00004: loss improved from 12509.62305 to 12343.49316, saving model to ./chkpt1
29/29 [==============================] - 1s 39ms/step - loss: 12343.4932
Epoch 5/10
28/29 [===========================>..] - ETA: 0s - loss: 12299.8691

In [4]:

start = time.time()
ranks = model.evaluate('/home/spai/code/ampligraph_projects/dataset/fb15k-237/test.txt', 
                       batch_size=100,
                       corrupt_side='s,o',
         use_filter={'train':'/home/spai/code/ampligraph_projects/dataset/fb15k-237/train.txt',
                  'valid':'/home/spai/code/ampligraph_projects/dataset/fb15k-237/valid.txt',
                  'test':'/home/spai/code/ampligraph_projects/dataset/fb15k-237/test.txt'})
end = time.time()
print('Time taken:', end-start)
mr_score(ranks), mrr_score(ranks), hits_at_n_score(ranks, 1), hits_at_n_score(ranks, 10)


28 triples containing invalid keys skipped!

9 triples containing invalid keys skipped!

28 triples containing invalid keys skipped!
206/206 [==============================] - 95s 463ms/step
Time taken: 104.37223482131958


(329.3355514238184,
 0.2040335050586587,
 0.12833936784421177,
 0.35475584695175655)

## Load Checkpoint and evaluate

In [5]:
start = time.time()
loaded_model = ScoringBasedEmbeddingModel(eta=50, 
                                     k=300, 
                                     scoring_type='TransE')
loaded_model.load_weights('./chkpt1')
ranks = loaded_model.evaluate('/home/spai/code/ampligraph_projects/dataset/fb15k-237/test.txt', 
                       batch_size=100,
                       corrupt_side='s,o',
         use_filter={'train':'/home/spai/code/ampligraph_projects/dataset/fb15k-237/train.txt',
                  'valid':'/home/spai/code/ampligraph_projects/dataset/fb15k-237/valid.txt',
                  'test':'/home/spai/code/ampligraph_projects/dataset/fb15k-237/test.txt'})
end = time.time()
print('Time taken:', end-start)
mr_score(ranks), mrr_score(ranks), hits_at_n_score(ranks, 1), hits_at_n_score(ranks, 10)


28 triples containing invalid keys skipped!

9 triples containing invalid keys skipped!

28 triples containing invalid keys skipped!
206/206 [==============================] - 93s 451ms/step
Time taken: 101.86448121070862


(329.3355514238184,
 0.2040335050586587,
 0.12833936784421177,
 0.35475584695175655)

# Training/eval with partition 

## Training with RandomEdges partitioner

In [6]:
from ampligraph.datasets import DummyBackend, SQLiteAdapter
from ampligraph.datasets import GraphDataLoader
from ampligraph.datasets.graph_partitioner import PARTITION_ALGO_REGISTRY
dataset_loader = GraphDataLoader('/home/spai/code/ampligraph_projects/dataset/fb15k-237/train.txt', 
                                 backend=SQLiteAdapter,
                                    batch_size=1000, 
                                    dataset_type='train', 
                                    epochs=10,
                                    use_indexer=True)

# Choose the partitioner 
partitioner = PARTITION_ALGO_REGISTRY.get('RandomEdges')(dataset_loader, k=3)

_split: memory before: 0.0Bytes, after: 12.96MB, consumed: 12.96MB; exec time: 35.535s


In [7]:
optim = tf.optimizers.Adam(learning_rate=0.001, amsgrad=True)

partitioned_model = ScoringBasedEmbeddingModel(eta=5, 
                                     k=300, 
                                     scoring_type='TransE')
partitioned_model.compile(optimizer=optim, loss=nll)

In [8]:

start = time.time()
partitioned_model.fit(partitioner,
                     batch_size=1000, use_partitioning=True,
                     epochs=10)
print((time.time()-start))

Epoch 1/10
274/274 [==============================] - 15s 56ms/step - loss: 1579.7837
Epoch 2/10
274/274 [==============================] - 15s 53ms/step - loss: 1403.8385
Epoch 3/10
274/274 [==============================] - 15s 55ms/step - loss: 1248.0869
Epoch 4/10
274/274 [==============================] - 15s 55ms/step - loss: 1118.7273
Epoch 5/10
274/274 [==============================] - 15s 55ms/step - loss: 1014.0121
Epoch 6/10
274/274 [==============================] - 15s 55ms/step - loss: 929.1785
Epoch 7/10
274/274 [==============================] - 15s 54ms/step - loss: 859.3276
Epoch 8/10
274/274 [==============================] - 15s 55ms/step - loss: 801.1460
Epoch 9/10
274/274 [==============================] - 15s 55ms/step - loss: 751.7992
Epoch 10/10
274/274 [==============================] - 15s 54ms/step - loss: 709.6334
193.23527336120605


In [9]:

dataset_loader_test = GraphDataLoader('/home/spai/code/ampligraph_projects/dataset/fb15k-237/test.txt', 
                                        backend=SQLiteAdapter,
                                        batch_size=400, 
                                        dataset_type='test', 
                                        epochs=1,
                                        use_indexer=partitioned_model.data_handler.get_mapper())

start = time.time()
ranks = partitioned_model.evaluate(dataset_loader_test, 
                                   batch_size=400)
end = time.time()
print('Time taken:', end-start)

mr_score(ranks), mrr_score(ranks), hits_at_n_score(ranks, 1), hits_at_n_score(ranks, 10)


28 triples containing invalid keys skipped!
52/52 [==============================] - 136s 3s/step
Time taken: 138.81118083000183


(504.7524019607843, 0.09983318850092686, 0.0, 0.27379901960784314)

## Save and Load

In [10]:
partitioned_model.save_weights('./best_model')

In [11]:
loaded_part_model = ScoringBasedEmbeddingModel(eta=5, 
                                     k=300, 
                                     scoring_type='TransE')

loaded_part_model.load_weights('./best_model')

In [12]:

dataset_loader_test = GraphDataLoader('/home/spai/code/ampligraph_projects/dataset/fb15k-237/test.txt', 
                                        backend=SQLiteAdapter,
                                        batch_size=400, 
                                        dataset_type='test', 
                                        epochs=1,
                                        use_indexer=loaded_part_model.data_indexer)

start = time.time()
ranks = loaded_part_model.evaluate(dataset_loader_test, 
                                   batch_size=400)
end = time.time()
print('Time taken:', end-start)

mr_score(ranks), mrr_score(ranks), hits_at_n_score(ranks, 1), hits_at_n_score(ranks, 10)


28 triples containing invalid keys skipped!
52/52 [==============================] - 143s 3s/step
Time taken: 146.41310167312622


(504.7524019607843, 0.09983318850092686, 0.0, 0.27379901960784314)

# Training/eval with partition (default Partitioning Approach)

In [3]:
from ampligraph.datasets import DummyBackend, SQLiteAdapter
from ampligraph.datasets import GraphDataLoader
from ampligraph.datasets.graph_partitioner import PARTITION_ALGO_REGISTRY


In [4]:
optim = tf.optimizers.Adam(learning_rate=0.001, amsgrad=True)

partitioned_model = ScoringBasedEmbeddingModel(eta=5, 
                                     k=300, 
                                     scoring_type='TransE')
partitioned_model.compile(optimizer=optim, loss=nll)


start = time.time()
partitioned_model.fit('/home/spai/code/ampligraph_projects/dataset/fb15k-237/train.txt',
                     batch_size=1000, use_partitioning=True,
                     epochs=10)
print((time.time()-start))

_split: memory before: 896.0Bytes, after: 12.904MB, consumed: 12.903MB; exec time: 146.49s
Epoch 1/10
    277/Unknown - 5s 20ms/step - loss: 1689.7427
Epoch 00001: loss improved from inf to 1689.05029, saving model to ./chkpt1
279/279 [==============================] - 6s 20ms/step - loss: 1689.0503
Epoch 2/10
277/279 [============================>.] - ETA: 0s - loss: 1590.5535
Epoch 00002: loss improved from 1689.05029 to 1590.34705, saving model to ./chkpt1
279/279 [==============================] - 5s 17ms/step - loss: 1590.3470
Epoch 3/10
277/279 [============================>.] - ETA: 0s - loss: 1481.9143
Epoch 00003: loss improved from 1590.34705 to 1481.84058, saving model to ./chkpt1
279/279 [==============================] - 5s 18ms/step - loss: 1481.8406
Epoch 4/10
277/279 [============================>.] - ETA: 0s - loss: 1380.2938
Epoch 00004: loss improved from 1481.84058 to 1380.22937, saving model to ./chkpt1
279/279 [==============================] - 5s 18ms/step - loss

In [5]:


start = time.time()
ranks = partitioned_model.evaluate('/home/spai/code/ampligraph_projects/dataset/fb15k-237/test.txt', 
                                   batch_size=400)
end = time.time()
print('Time taken:', end-start)

mr_score(ranks), mrr_score(ranks), hits_at_n_score(ranks, 1), hits_at_n_score(ranks, 10)


28 triples containing invalid keys skipped!
53/53 [==============================] - 169s 3s/step
Time taken: 173.1760048866272


(703.0901996281436, 0.0938227619140394, 0.0, 0.2562383794891868)

## Filtered evaluation

In [6]:
start = time.time()
ranks = partitioned_model.evaluate('/home/spai/code/ampligraph_projects/dataset/fb15k-237/test.txt', 
                       batch_size=200,
                       corrupt_side='s,o',
                        use_filter={'train':'/home/spai/code/ampligraph_projects/dataset/fb15k-237/train.txt',
                              'valid':'/home/spai/code/ampligraph_projects/dataset/fb15k-237/valid.txt',
                              'test':'/home/spai/code/ampligraph_projects/dataset/fb15k-237/test.txt'})
end = time.time()
print('Time taken:', end-start)
mr_score(ranks), mrr_score(ranks), hits_at_n_score(ranks, 1), hits_at_n_score(ranks, 10)


28 triples containing invalid keys skipped!

9 triples containing invalid keys skipped!

28 triples containing invalid keys skipped!
104/104 [==============================] - 1097s 11s/step
Time taken: 1193.5984427928925


(577.7298659360016,
 0.21288052721907064,
 0.14925628730795576,
 0.33736177708190623)